In [ ]:
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

from sklearn.metrics import precision_recall_fscore_support

# <a id='1'>1. Confusion matrix</a>

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1]
y_pred = [0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1]
conf_matrix = confusion_matrix(y_true, y_pred)

# Diag: TN [0,0], TP [1,1] 
# off-Diag: FN [0,1], FP [1,0]
conf_matrix

In [ ]:
# true Churn - pred Churn -- TP: 7
# true non Churn - pred non Churn -- TN: 6
# true non Churn - pred Churn -- FP: 2
# true Churn - pred non Churn -- FN: 8
# [7, 8]
# [2, 6]
conf_matrix_disp = conf_matrix[:, ::-1]
#conf_matrix = conf_matrix[::-1, :]
conf_matrix_disp

In [ ]:
#plot confusion matrix
conf_matrix_disp_plot = go.Heatmap(z = conf_matrix_disp ,
                        x = ["(pred) Churn","(pred) Not churn"],
                        y = ["(true) Not churn","(true) Churn"],
                        showscale  = False, colorscale = "Picnic",
                        name = "matrix")

In [ ]:
fig = go.Figure(
    data=[conf_matrix_disp_plot],
    layout_title_text="Confusion matrix"
)
py.iplot(fig)

# <a id='2'>2. Precision, Recall and Specificity</a>

In [ ]:
# precision; TP/(TP+FP)
print("Precision {:.4f}".format(conf_matrix[1,1]/(conf_matrix[1,1]+conf_matrix[1,0])))
# recall; TP/(TP+FN)
print("Recall {:.4f}".format(conf_matrix[1,1]/(conf_matrix[1,1]+conf_matrix[0,1])))
# specificity; TN/(TN+FP)
print("Specificity {:.4f}".format(conf_matrix[1,0]/(conf_matrix[1,0]+conf_matrix[0,0])))

# <a id='3'>3. ROC curve</a>

In [ ]:
# Create an arbitrary data 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# generate two class dataset
X, y = make_classification(n_samples=1000, n_classes=2, 
                           n_features=20, random_state=27)

# split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=27)


In [ ]:
# Test the performance of two classifiers

# train models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# logistic regression
model1 = LogisticRegression()
# knn
model2 = KNeighborsClassifier(n_neighbors=4)

# fit model
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)

# predict probabilities
pred_prob1 = model1.predict_proba(X_test)
pred_prob2 = model2.predict_proba(X_test)

In [ ]:
from sklearn.metrics import roc_curve

# roc curve for models
fpr1, tpr1, thresh1 = roc_curve(y_test, pred_prob1[:,1], pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(y_test, pred_prob2[:,1], pos_label=1)

# roc curve for tpr = fpr 
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
# Plot the ROC curve
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# plot roc curves
plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='Logistic Regression')
plt.plot(fpr2, tpr2, linestyle='--',color='green', label='KNN')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

# <a id='4'>4. AUC</a>

In [ ]:
from sklearn.metrics import roc_auc_score

# auc scores
auc_score1 = roc_auc_score(y_test, pred_prob1[:,1])
auc_score2 = roc_auc_score(y_test, pred_prob2[:,1])

print("{:.4f}".format(auc_score1), "{:.4f}".format(auc_score2))

In [ ]:
#plot roc curve
LR_trace = go.Scatter(x = fpr1,y = tpr1,
                    name = "Roc : " + "{:.4f}".format(auc_score1),
                    line = dict(color = ('rgb(22, 96, 167)'),width = 2))
KNN_trace = go.Scatter(x = fpr2,y = tpr2,
                    name = "Roc : " + "{:.4f}".format(auc_score2),
                    line = dict(color = ('rgb(22, 96, 0)'),width = 2))
random_trace = go.Scatter(x = [0,1],y=[0,1],
                    line = dict(color = ('rgb(205, 12, 24)'),
                                width = 2,
                    dash = 'dot'))

py.iplot([LR_trace, KNN_trace, random_trace])